In [1]:
import os
import torch
import skimage
import pywt
import scipy.io
import scipy.signal
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt
from scipy import stats
from einops import reduce, rearrange, repeat
from npeet import entropy_estimators as ee
from torch.optim.lr_scheduler import StepLR
from scipy.fft import rfft, rfftfreq, ifft
from einops import rearrange
from torch_geometric.data import InMemoryDataset, Data, DataLoader
from Electrodes import Electrodes
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

In [2]:
class DEAPDatasetEEGFeatures(InMemoryDataset):
  def __init__(self, root, raw_dir, processed_dir, feature='de',target='participant_id', transform=None, pre_transform=None,include_edge_attr = True, undirected_graphs = True, add_global_connections=True, participant_from=1, participant_to=32, n_videos=40):
      self._raw_dir = raw_dir
      self._processed_dir = processed_dir
      self.participant_from = participant_from
      self.participant_to = participant_to
      self.n_videos = n_videos
      self.feature = feature
      self.target = target
      # Whether or not to include edge_attr in the dataset
      self.include_edge_attr = include_edge_attr
      # If true there will be 1024 links as opposed to 528
      self.undirected_graphs = undirected_graphs
      # Instantiate class to handle electrode positions
      print('Using global connections' if add_global_connections else 'Not using global connections')
      self.electrodes = Electrodes(add_global_connections, expand_3d = False)
      super(DEAPDatasetEEGFeatures, self).__init__(root, transform, pre_transform)
      self.data, self.slices = torch.load(self.processed_paths[0])
      
  @property
  def raw_dir(self):
      return f'{self.root}/{self._raw_dir}'

  @property
  def processed_dir(self):
      return f'{self.root}/{self._processed_dir}'

  @property
  def raw_file_names(self):
      raw_names = [f for f in os.listdir(self.raw_dir)]
      raw_names.sort()
      return raw_names

  @property
  def processed_file_names(self):
      if not os.path.exists(self.processed_dir):
        os.makedirs(self.processed_dir)
      file_name = f'{self.participant_from}-{self.participant_to}' if self.participant_from is not self.participant_to else f'{self.participant_from}'
      return [f'deap_processed_graph.{file_name}_{self.feature}_{self.target}.dataset']

  def process(self):
        # Number of nodes per graph
        n_nodes = len(self.electrodes.positions_3d)
        

        if self.undirected_graphs:
            source_nodes, target_nodes = np.repeat(np.arange(0,n_nodes),n_nodes), np.tile(np.arange(0,n_nodes),n_nodes)
        else:
            source_nodes, target_nodes = np.tril_indices(n_nodes,n_nodes)
        
        edge_attr = self.electrodes.adjacency_matrix[source_nodes,target_nodes]
        
        # Remove zero weight links
        mask = np.ma.masked_not_equal(edge_attr, 0).mask
        edge_attr,source_nodes,target_nodes = edge_attr[mask], source_nodes[mask], target_nodes[mask]

        edge_attr, edge_index = torch.FloatTensor(edge_attr), torch.tensor([source_nodes,target_nodes], dtype=torch.long)
        
        # Expand edge_index and edge_attr to match windows
        e_edge_index = edge_index.clone()
        e_edge_attr = edge_attr.clone()
        
        number_of_graphs = 4
        for i in range(number_of_graphs-1):
            a = edge_index + e_edge_index.max() + 1
            e_edge_index = torch.cat([e_edge_index,a],dim=1)
            e_edge_attr = torch.cat([e_edge_attr,edge_attr],dim=0)

        print(f'Number of graphs per video: {number_of_graphs}')
        # List of graphs that will be written to file
        data_list = []
        pbar = tqdm(range(self.participant_from,self.participant_to+1))
        for participant_id in pbar:
            raw_name = [e for e in self.raw_file_names if str(participant_id).zfill(2) in e][0]
            pbar.set_description(raw_name)
            # Load raw file as np array
            participant_data = scipy.io.loadmat(f'{self.raw_dir}/{raw_name}')
            signal_data = torch.FloatTensor(remove_baseline_mean(participant_data['data'][:,:32,:]))
#             signal_data = torch.FloatTensor()
            processed = []
            for video_index, video in enumerate(signal_data[:self.n_videos,:,:]):
                if self.feature == 'wav':
                    node_features = process_video_wavelet(video)
                elif self.feature =='wav-entropy':
                    node_features = process_video_wavelet(video, feature='entropy')
                else:
                    node_features = process_video(video, feature=self.feature)
                
                if self.target == 'emotion_labels':
                    target = [participant_data['labels'][video_index]]
                if self.target == 'participant_id':
                    target = participant_id-1
                elif self.target == 'video_id':
                    target = video_index
                else:
                    raise 'Invalid target'
                data = Data(x=torch.FloatTensor(node_features),edge_attr=e_edge_attr,edge_index=e_edge_index, y=torch.LongTensor([target])) if self.include_edge_attr else Data(x=torch.FloatTensor(node_features), edge_index=e_edge_index, y=torch.LongTensor([target]))
                data_list.append(data) 
               
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [3]:
def calculate_de(window):
    return ee.entropy(window.reshape(-1,1), k=2)
# Input: Video with shape (32,7680)
# Output: Graph node features with shape (5*32, 59) -> 5 graphs with 32 nodes each with 59 features each
def process_video(video, feature='psd'):
    # Transform to frequency domain
    fft_vals = np.fft.rfft(video, axis=-1)
     # Get frequencies for amplitudes in Hz
    samplingFrequency = 128
    fft_freq = np.fft.rfftfreq(video.shape[-1], 1.0/samplingFrequency)
    # Delta, Theta, Alpha, Beta, Gamma
    bands = [(0,4),(4,8),(8,12),(12,30),(30,45)]
    
    band_mask = np.array([np.logical_or(fft_freq < f, fft_freq > t) for f,t in bands])
    band_mask = repeat(band_mask,'a b -> a c b', c=32)
    band_data = np.array(fft_vals)
    band_data = repeat(band_data,'a b -> c a b', c=5)
     
    band_data[band_mask] = 0
    
    band_data = np.fft.irfft(band_data)

    windows = skimage.util.view_as_windows(band_data, (5,32,128), step=128).squeeze()
    # (5, 32, 60, 128)
    windows = rearrange(windows, 'a b c d -> b c a d')
    
    if feature == 'psd':
        features = scipy.signal.periodogram(windows)[1]
        features = np.mean(features, axis=-1)
    elif feature == 'de':
        features = np.apply_along_axis(calculate_de, -1, windows)

    
    features = rearrange(features, 'a b c -> (a b) c')
    features = torch.FloatTensor(features)

    return features

In [4]:
def remove_baseline_mean(signal_data):
    # Take first three senconds of data
    signal_baseline = np.array(signal_data[:,:,:128*3]).reshape(40,32,128,-1)
    # Mean of three senconds of baseline will be deducted from all windows
    signal_noise = np.mean(signal_baseline,axis=-1)
    # Expand mask
    signal_noise = repeat(signal_noise,'a b c -> a b (d c)',d=60)
    return signal_data[:,:,128*3:] - signal_noise

In [5]:
def process_video_wavelet(video, feature='energy', time_domain=False):
    band_widths = [32,16,8,4]
    features = []
    for i in range(5):
        if i == 0:
            # Highest frequencies (64-128Hz) are not used
            cA, cD = pywt.dwt(video.numpy(), 'db4')
        else:
            cA, cD = pywt.dwt(cA, 'db4')
            
            cA_windows = skimage.util.view_as_windows(cA, (32,band_widths[i-1]*2), step=band_widths[i-1]).squeeze()
            cA_windows = np.transpose(cA_windows[:59,:,:],(1,0,2))
            if feature == 'energy':
                cA_windows = np.square(cA_windows)
                cA_windows = np.sum(cA_windows, axis=-1)
                features.append(cA_windows)
            elif feature == 'entropy':
                cA_windows = np.square(cA_windows) * np.log(np.square(cA_windows))
                cA_windows = -np.sum(cA_windows, axis=-1)
                features.append(cA_windows)

            else:
                raise 'Error, invalid wavelet feature'
                
    if time_domain:
        features = np.transpose(features,(2,1,0))
    features = rearrange(features, 'a b c -> (a b) c')
    features = torch.FloatTensor(features)
    
    # Normalization
    m = features.mean(0, keepdim=True)
    s = features.std(0, unbiased=False, keepdim=True)
    features -= m
    features /= s
    return features

In [6]:
# Constants used to define data paths
ROOT_DIR = './'
RAW_DIR = 'data/matlabPREPROCESSED'
PROCESSED_DIR = 'data/graphProcessedData'

dataset = DEAPDatasetEEGFeatures(root= ROOT_DIR, raw_dir= RAW_DIR, processed_dir= PROCESSED_DIR, feature='wav',participant_to=32)
# dataset = dataset.shuffle()

Using global connections


In [7]:
# 880 used for training, 220 validation and 180 testing
# test_participant = 1
# 
splt_idx = 1100

dataset = dataset.shuffle()

# 85% used for train/val
train_dataset = dataset[:splt_idx]
test_dataset = dataset[splt_idx:]

len(train_dataset),len(test_dataset)

(1100, 180)

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
print(device)

cuda


In [9]:
n_classes = np.unique(np.array([d.y for d in dataset])).shape[0]
n_classes

32

In [26]:
from torch_geometric.nn import GCN2Conv, GraphConv, global_max_pool as gmp
class Model(torch.nn.Module):
    def __init__(self, in_channels,n_graphs, hidden_channels=16, n_classes = 32):
        super(Model, self).__init__()
        
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        
        self.gconv1 = GraphConv(in_channels,hidden_channels*2)
        self.gconv2 = GraphConv(hidden_channels*2,hidden_channels)
        
#         self.gconv3 = GCNConv(in_channels,hidden_channels)
        
        # self.rnn = torch.nn.GRU(hidden_channels, rnn_hidden_dim, 2,dropout=0.2, batch_first=True)
        self.cnn1 = torch.nn.Conv1d(n_graphs, 1, kernel_size=3, stride=2)
        
        self.lin1 = torch.nn.Linear(224, hidden_channels)
        self.lin2 = torch.nn.Linear(hidden_channels, n_classes)

        self.softmax = nn.Softmax(dim=-1)

        
    def forward(self, batch):
        bs = len(torch.unique(batch.batch))
        x, edge_index, edge_attr = batch.x, batch.edge_index, batch.edge_attr
#         print(x.shape)
        x = self.gconv1(x, edge_index, edge_attr)
        x = self.gconv2(x, edge_index, edge_attr)
        x = F.dropout(x, p=0.4, training=self.training)
        x = x.relu()
#         print(x.shape)
        x = rearrange(x, '(bs g e) f -> (bs e) g f', bs=bs, e=32)
#         print(x.shape)
        x = self.cnn1(x).squeeze()
        x = x.tanh()
        x = rearrange(x, '(bs e) f -> bs (e f)', bs=bs)
#         print(x.shape)
#         x = torch.sum(x, dim=1)
        
        
        x = F.dropout(x, p=0.3, training=self.training)
#         print(x.shape)
        x = self.lin1(x)
        x = x.relu()
        x = self.lin2(x)
#         print(x.shape)
#         x = x.view(-1)
       
        x = self.softmax(x)
        return x

        

In [11]:
train_dataset[0].x.shape[1]

59

In [ ]:
# %%timeit

model = Model(train_dataset[0].x.shape[1],train_dataset[0].x.shape[0]//32, n_classes=n_classes).to(device)  
pytorch_total_params = sum(p.numel() for p in model.parameters())
print(f'Model parameter count: {pytorch_total_params}')

# model = model.to(devic)
# optimizer = torch.optim.Adadelta(model.parameters(), lr=.1, rho=0.9, eps=1e-06, weight_decay=1e-5)
# optimizer = torch.optim.SGD(model.parameters(),lr=1e-2, weight_decay=0)
optimizer = torch.optim.Adam(model.parameters(),lr=2e-3, weight_decay=0)
# optimizer = torch.optim.Adam(model.parameters())
# optimizer = torch.optim.Adagrad(model.parameters(), lr=1e-5)
# optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001, weight_decay=5e-4)
# optimizer = torch.optim.Adagrad(model.parameters(), lr=5e-4, lr_decay=0, weight_decay=0)

# Instantiate optimizer
# scheduler = StepLR(optimizer, step_size=20, gamma=0.7)

# criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()



def train(loader, target = 0):
    model.train()
    losses = []
    right = 0
    tot = 0
    for batch in loader:
        optimizer.zero_grad()
        batch = batch.to(device)
        out = model(batch)
        loss = criterion(out,batch.y)
        loss.backward()
        losses.append(loss.item())
        optimizer.step()
        pred = torch.argmax(out,-1)
        right += torch.sum((pred == batch.y).detach().cpu())
        tot += batch.y.shape[0]
        
    return np.array(losses).mean(), right/tot

def test(loader,verbose=False, target = 0):
    model.eval()
    losses = []
    right = 0
    tot = 0
    for batch in loader:
        batch = batch.to(device)
        # y = batch.y[:,target]
        out = model(batch)
        pred = torch.argmax(out,-1)
        if verbose:
            print(pred,batch.y)
        loss = criterion(out,batch.y)
        losses.append(loss.item())
        
        right += torch.sum((pred == batch.y).detach().cpu())
        tot += batch.y.shape[0]
    return np.array(losses).mean(), right/tot

best_val_loss = np.inf
esp = 0
MAX_ESP = 50

BS = 16

target = 0 # Valence-Arousal-Dominance-Liking

splt_idx = 1000
train_data, val_data = torch.utils.data.random_split(train_dataset, [splt_idx, len(train_dataset)-splt_idx])

train_loader = DataLoader(train_data, batch_size=BS, shuffle=True)
val_loader = DataLoader(val_data, batch_size=BS)
writer = SummaryWriter()
for epoch in range(1, 10000):    

    
    
    # Training and validation
    train_loss, train_acc = train(train_loader, target = target)
    val_loss, val_acc = test(val_loader , target = target)
    print(f'Epoch {epoch};t loss: {train_loss:.5f} ;t acc: {train_acc:.2f} ;v loss: {val_loss:.5f} ;v acc: {val_acc:.2f}')

    writer.add_scalar('Loss/train', train_loss, epoch)
    writer.add_scalar('Loss/test', val_loss, epoch)
    writer.add_scalar('Accuracy/train', train_acc, epoch)
    writer.add_scalar('Accuracy/test', val_acc, epoch)
    # Early stopping and checkpoint
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        esp = 0
        torch.save(model.state_dict(),'./best_params') 
    else:
        esp += 1
        if esp >= MAX_ESP:
            break
            
    if epoch % 20 == 0:
        test_loader = DataLoader(test_dataset, batch_size=2)
        loss, acc = test(test_loader, True)
        print(f'Test loss: {loss} ; Test acc: {acc}')
        
    


print('Finished training')

Model parameter count: 9005
Epoch 1;t loss: 3.45806 ;t acc: 0.06 ;v loss: 3.43867 ;v acc: 0.05
Epoch 2;t loss: 3.35289 ;t acc: 0.20 ;v loss: 3.28314 ;v acc: 0.26
Epoch 3;t loss: 3.19496 ;t acc: 0.38 ;v loss: 3.15777 ;v acc: 0.42
Epoch 4;t loss: 3.06287 ;t acc: 0.50 ;v loss: 3.04407 ;v acc: 0.54
Epoch 5;t loss: 2.94194 ;t acc: 0.63 ;v loss: 2.87982 ;v acc: 0.67
Epoch 6;t loss: 2.87515 ;t acc: 0.68 ;v loss: 2.87171 ;v acc: 0.67
Epoch 7;t loss: 2.84066 ;t acc: 0.71 ;v loss: 2.84002 ;v acc: 0.70
Epoch 8;t loss: 2.80321 ;t acc: 0.74 ;v loss: 2.75127 ;v acc: 0.79
Epoch 9;t loss: 2.75255 ;t acc: 0.80 ;v loss: 2.69699 ;v acc: 0.83
Epoch 10;t loss: 2.72624 ;t acc: 0.81 ;v loss: 2.69622 ;v acc: 0.83
Epoch 11;t loss: 2.71671 ;t acc: 0.81 ;v loss: 2.69545 ;v acc: 0.83
Epoch 12;t loss: 2.71194 ;t acc: 0.81 ;v loss: 2.68324 ;v acc: 0.85
Epoch 13;t loss: 2.69010 ;t acc: 0.84 ;v loss: 2.66004 ;v acc: 0.87
Epoch 14;t loss: 2.68389 ;t acc: 0.84 ;v loss: 2.66012 ;v acc: 0.87
Epoch 15;t loss: 2.67951 ;t a

Epoch 31;t loss: 2.58112 ;t acc: 0.94 ;v loss: 2.60696 ;v acc: 0.93
Epoch 32;t loss: 2.57757 ;t acc: 0.94 ;v loss: 2.60685 ;v acc: 0.93
Epoch 33;t loss: 2.57873 ;t acc: 0.94 ;v loss: 2.60703 ;v acc: 0.93
Epoch 34;t loss: 2.57816 ;t acc: 0.94 ;v loss: 2.60694 ;v acc: 0.93
Epoch 35;t loss: 2.57998 ;t acc: 0.94 ;v loss: 2.60660 ;v acc: 0.93
Epoch 36;t loss: 2.57777 ;t acc: 0.94 ;v loss: 2.60668 ;v acc: 0.93
Epoch 37;t loss: 2.57562 ;t acc: 0.94 ;v loss: 2.60602 ;v acc: 0.93
Epoch 38;t loss: 2.57620 ;t acc: 0.94 ;v loss: 2.60648 ;v acc: 0.93
Epoch 39;t loss: 2.57745 ;t acc: 0.94 ;v loss: 2.60652 ;v acc: 0.93
Epoch 40;t loss: 2.57611 ;t acc: 0.94 ;v loss: 2.60620 ;v acc: 0.93
tensor([24,  8], device='cuda:0') tensor([24,  8], device='cuda:0')
tensor([11,  9], device='cuda:0') tensor([11,  9], device='cuda:0')
tensor([ 9, 16], device='cuda:0') tensor([ 5, 16], device='cuda:0')
tensor([25,  9], device='cuda:0') tensor([25,  5], device='cuda:0')
tensor([18, 19], device='cuda:0') tensor([ 5, 19

Epoch 61;t loss: 2.57523 ;t acc: 0.94 ;v loss: 2.60626 ;v acc: 0.93
Epoch 62;t loss: 2.57293 ;t acc: 0.95 ;v loss: 2.53642 ;v acc: 0.98
Epoch 63;t loss: 2.55676 ;t acc: 0.97 ;v loss: 2.53597 ;v acc: 0.98
Epoch 64;t loss: 2.55312 ;t acc: 0.97 ;v loss: 2.53578 ;v acc: 0.98
Epoch 65;t loss: 2.55032 ;t acc: 0.97 ;v loss: 2.53575 ;v acc: 0.98
Epoch 66;t loss: 2.54841 ;t acc: 0.97 ;v loss: 2.53566 ;v acc: 0.98
Epoch 67;t loss: 2.54746 ;t acc: 0.97 ;v loss: 2.53568 ;v acc: 0.98
Epoch 68;t loss: 2.54612 ;t acc: 0.97 ;v loss: 2.53572 ;v acc: 0.98
Epoch 69;t loss: 2.54617 ;t acc: 0.97 ;v loss: 2.53573 ;v acc: 0.98
Epoch 70;t loss: 2.54690 ;t acc: 0.97 ;v loss: 2.53566 ;v acc: 0.98
Epoch 71;t loss: 2.54605 ;t acc: 0.97 ;v loss: 2.53569 ;v acc: 0.98
Epoch 72;t loss: 2.54652 ;t acc: 0.97 ;v loss: 2.53567 ;v acc: 0.98
Epoch 73;t loss: 2.54604 ;t acc: 0.97 ;v loss: 2.53568 ;v acc: 0.98
Epoch 74;t loss: 2.54662 ;t acc: 0.97 ;v loss: 2.53564 ;v acc: 0.98
Epoch 75;t loss: 2.54594 ;t acc: 0.97 ;v loss: 2

Epoch 92;t loss: 2.52111 ;t acc: 1.00 ;v loss: 2.52281 ;v acc: 1.00
Epoch 93;t loss: 2.51982 ;t acc: 1.00 ;v loss: 2.52483 ;v acc: 0.99
Epoch 94;t loss: 2.52058 ;t acc: 1.00 ;v loss: 2.52391 ;v acc: 1.00
Epoch 95;t loss: 2.52265 ;t acc: 1.00 ;v loss: 2.52581 ;v acc: 0.99
Epoch 96;t loss: 2.52182 ;t acc: 1.00 ;v loss: 2.52675 ;v acc: 0.99
Epoch 97;t loss: 2.52190 ;t acc: 1.00 ;v loss: 2.52607 ;v acc: 0.99
Epoch 98;t loss: 2.52181 ;t acc: 1.00 ;v loss: 2.52589 ;v acc: 0.99
Epoch 99;t loss: 2.52516 ;t acc: 0.99 ;v loss: 2.52685 ;v acc: 0.99
Epoch 100;t loss: 2.52165 ;t acc: 1.00 ;v loss: 2.52636 ;v acc: 0.99
tensor([24,  8], device='cuda:0') tensor([24,  8], device='cuda:0')
tensor([11,  9], device='cuda:0') tensor([11,  9], device='cuda:0')
tensor([ 5, 16], device='cuda:0') tensor([ 5, 16], device='cuda:0')
tensor([25,  5], device='cuda:0') tensor([25,  5], device='cuda:0')
tensor([ 5, 19], device='cuda:0') tensor([ 5, 19], device='cuda:0')
tensor([ 7, 17], device='cuda:0') tensor([ 7, 1

tensor([19, 23], device='cuda:0') tensor([19, 23], device='cuda:0')
tensor([25, 14], device='cuda:0') tensor([25, 14], device='cuda:0')
tensor([10,  9], device='cuda:0') tensor([10,  9], device='cuda:0')
tensor([13, 25], device='cuda:0') tensor([13, 25], device='cuda:0')
tensor([27,  8], device='cuda:0') tensor([27,  8], device='cuda:0')
tensor([11, 29], device='cuda:0') tensor([11, 29], device='cuda:0')
Test loss: 2.5180796252356634 ; Test acc: 1.0
Epoch 121;t loss: 2.51897 ;t acc: 1.00 ;v loss: 2.52420 ;v acc: 0.99
Epoch 122;t loss: 2.51927 ;t acc: 1.00 ;v loss: 2.52403 ;v acc: 0.99
Epoch 123;t loss: 2.51828 ;t acc: 1.00 ;v loss: 2.52351 ;v acc: 0.99
Epoch 124;t loss: 2.51893 ;t acc: 1.00 ;v loss: 2.52696 ;v acc: 0.99
Epoch 125;t loss: 2.52007 ;t acc: 1.00 ;v loss: 2.52654 ;v acc: 0.99
Epoch 126;t loss: 2.52176 ;t acc: 1.00 ;v loss: 2.51849 ;v acc: 1.00
Epoch 127;t loss: 2.52262 ;t acc: 1.00 ;v loss: 2.52545 ;v acc: 0.99
Epoch 128;t loss: 2.52087 ;t acc: 1.00 ;v loss: 2.52690 ;v acc:

Epoch 145;t loss: 2.51834 ;t acc: 1.00 ;v loss: 2.51823 ;v acc: 1.00
Epoch 146;t loss: 2.51838 ;t acc: 1.00 ;v loss: 2.51808 ;v acc: 1.00
Epoch 147;t loss: 2.51953 ;t acc: 1.00 ;v loss: 2.51866 ;v acc: 1.00
Epoch 148;t loss: 2.52212 ;t acc: 1.00 ;v loss: 2.51804 ;v acc: 1.00
Epoch 149;t loss: 2.52092 ;t acc: 1.00 ;v loss: 2.51877 ;v acc: 1.00
Epoch 150;t loss: 2.52202 ;t acc: 1.00 ;v loss: 2.52689 ;v acc: 0.99
Epoch 151;t loss: 2.52083 ;t acc: 1.00 ;v loss: 2.52673 ;v acc: 0.99
Epoch 152;t loss: 2.52882 ;t acc: 0.99 ;v loss: 2.52673 ;v acc: 0.99
Epoch 153;t loss: 2.52313 ;t acc: 1.00 ;v loss: 2.52696 ;v acc: 0.99
Epoch 154;t loss: 2.52311 ;t acc: 1.00 ;v loss: 2.52689 ;v acc: 0.99
Epoch 155;t loss: 2.52579 ;t acc: 1.00 ;v loss: 2.52418 ;v acc: 0.99
Epoch 156;t loss: 2.52287 ;t acc: 1.00 ;v loss: 2.51874 ;v acc: 1.00
Epoch 157;t loss: 2.52031 ;t acc: 1.00 ;v loss: 2.52003 ;v acc: 1.00
Epoch 158;t loss: 2.51930 ;t acc: 1.00 ;v loss: 2.51806 ;v acc: 1.00
Epoch 159;t loss: 2.51906 ;t acc: 

Epoch 175;t loss: 2.51902 ;t acc: 1.00 ;v loss: 2.51932 ;v acc: 1.00
Epoch 176;t loss: 2.51820 ;t acc: 1.00 ;v loss: 2.51952 ;v acc: 1.00
Epoch 177;t loss: 2.51877 ;t acc: 1.00 ;v loss: 2.52111 ;v acc: 1.00
Epoch 178;t loss: 2.51903 ;t acc: 1.00 ;v loss: 2.52386 ;v acc: 0.99
Epoch 179;t loss: 2.51914 ;t acc: 1.00 ;v loss: 2.52690 ;v acc: 0.99
Epoch 180;t loss: 2.51809 ;t acc: 1.00 ;v loss: 2.52689 ;v acc: 0.99
tensor([24,  8], device='cuda:0') tensor([24,  8], device='cuda:0')
tensor([11,  9], device='cuda:0') tensor([11,  9], device='cuda:0')
tensor([ 5, 16], device='cuda:0') tensor([ 5, 16], device='cuda:0')
tensor([25,  5], device='cuda:0') tensor([25,  5], device='cuda:0')
tensor([ 5, 19], device='cuda:0') tensor([ 5, 19], device='cuda:0')
tensor([ 7, 17], device='cuda:0') tensor([ 7, 17], device='cuda:0')
tensor([29, 12], device='cuda:0') tensor([29, 12], device='cuda:0')
tensor([29,  9], device='cuda:0') tensor([29,  9], device='cuda:0')
tensor([27, 10], device='cuda:0') tensor([

In [32]:
model.load_state_dict(torch.load('./best_params'))
test_loader = DataLoader(test_dataset, batch_size=1)
loss, acc = test(train_loader, False,target=target)
print(f'Train loss: {loss} ; Train acc: {acc}')
loss, acc = test(val_loader, False,target=target)

print(f'Val loss: {loss} ; Val acc: {acc}')
loss, acc = test(test_loader, True,target=target)
print(f'Test loss: {loss} ; Test acc: {acc}')

Train loss: 2.635221943259239 ; Train acc: 0.8790000081062317
Val loss: 2.626517593860626 ; Val acc: 0.8600000143051147
tensor([24], device='cuda:0') tensor([24], device='cuda:0')
tensor([8], device='cuda:0') tensor([8], device='cuda:0')
tensor([11], device='cuda:0') tensor([11], device='cuda:0')
tensor([9], device='cuda:0') tensor([9], device='cuda:0')
tensor([5], device='cuda:0') tensor([5], device='cuda:0')
tensor([16], device='cuda:0') tensor([16], device='cuda:0')
tensor([25], device='cuda:0') tensor([25], device='cuda:0')
tensor([5], device='cuda:0') tensor([5], device='cuda:0')
tensor([5], device='cuda:0') tensor([5], device='cuda:0')
tensor([2], device='cuda:0') tensor([19], device='cuda:0')
tensor([7], device='cuda:0') tensor([7], device='cuda:0')
tensor([17], device='cuda:0') tensor([17], device='cuda:0')
tensor([29], device='cuda:0') tensor([29], device='cuda:0')
tensor([8], device='cuda:0') tensor([12], device='cuda:0')
tensor([29], device='cuda:0') tensor([29], device='cud